In [1]:
import pandas as pd
import numpy as np

In [2]:
def get_dicts():
    days_dict = {
        'D1':'Monday',
        'D2':'Tuesday',
        'D4':'Thursday',
        'D5':'Friday'
    }

    periods_dict = {
        'P1':{
            'start':'09:30',
            'end':'11:00'
        },
        'P2':{
        'start':'11:00',
        'end':'12:30'
        },
        'P3':{
        'start':'14:00',
        'end':'15:30'
        },
        'P4':{
        'start':'15:30',
        'end':'17:00'
        },
    }
    
    return days_dict, periods_dict

In [17]:
def get_df():
    raw_df = pd.read_csv('module_data_2025.csv')

    raw_df['class_no'] = ['-']*len(raw_df)

    df = pd.DataFrame(columns=('year_of_study','term','professor','module_name','class_no')) # Update to 'classes_per_week' in csv file

    for i in range(len(raw_df)):
        row = raw_df.loc[i]
        for c in range(row['classes_per_week']):
            df.loc[len(df)] = {
                'year_of_study':row['year_of_study'],
                'term':row['term'],
                'professor':row['professor'],
                'module_name':row['module_name'],
                'class_no':f'C{c}'
            }

    df = df[df['term']=='winter']
    df = df.drop(columns=['term'])
    return df

In [4]:
def calc_timetable(df):
    df['period'] = ['-']*len(df)
    availability = ['D1P1','D1P2','D1P3','D1P4','D2P1','D2P2','D2P3','D2P4','D4P1','D4P2','D4P3','D4P4','D5P1','D5P2','D5P3','D5P4']
    year_avs = [availability[:],availability[:],availability[:]]
    years = df['year_of_study'].unique()
    for p in df['professor'].unique():
        prof_av = availability[:]
        for y in range(len(years)):
            modules = df[(df['professor']==p) & (df['year_of_study']==years[y])]
            common_availability = list(set(prof_av) & set(year_avs[y]))
            for i in modules.index:
                if len(common_availability) == 0:
                    raise SystemExit('There are not enough teachers for these modules')
                else:
                    period = np.random.choice(common_availability)
                    df.loc[i,'period'] = period
                    prof_av.remove(period)
                    year_avs[y].remove(period)
                    common_availability.remove(period)
    return df

In [ ]:
def test_timetable(df):
    # Test teachers
    test = True
    for i in df['professor'].unique():
        prof_classes = df[df['professor']==i]
        d_sum = prof_classes['period'].duplicated().sum()
        # print(i,d_sum)
        if d_sum > 0:
            print(f'{prof_classes[['professor','period']]}\n')
            test = False
    if test == True:
        print('Staff test successful!')

    # Test students
    test = True
    for i in df['year_of_study'].unique():
        year_classes = df[df['year_of_study']==i]
        d_sum = prof_classes['period'].duplicated().sum()
        # print('Year',i,d_sum)
        if d_sum > 0:
            print(prof_classes[['year_of_study','period']])
            test = False
    if test == True:
        print('Students test successful!')


In [25]:
def present_timetable(df,days_dict,periods_dict):
    days_list = []
    start_list = []
    end_list = []
    for i in range(len(df)):
        days_list.append(days_dict[df.loc[i]['period'][:2]])
        start_list.append(periods_dict[df.loc[i]['period'][2:]]['start'])
        end_list.append(periods_dict[df.loc[i]['period'][2:]]['end'])
    df['day'] = days_list
    df['class_start'] = start_list
    df['class_end'] = end_list
    df = df.sort_values('period')
    df = df.reset_index(drop=True)
    to_show = df[['year_of_study','professor','module_name','day','class_start','class_end']]
    return to_show

In [27]:
days_dict, periods_dict = get_dicts()
df = get_df()
df = calc_timetable(df)
test_timetable(df)
df = present_timetable(df,days_dict,periods_dict)
df[(df['year_of_study']==2) & (df['module_name'].isin(['data_science','problems_2a','conceptual_tools']))]

Staff test successful!
Students test successful!


,year_of_study,professor,module_name,day,class_start,class_end
6,2,Ash,problems_2a,Monday,14:00,15:30
7,2,Waqas,conceptual_tools,Monday,15:30,17:00
18,2,Nicollo,data_science,Thursday,09:30,11:00
20,2,Nicollo,data_science,Thursday,11:00,12:30
23,2,Waqas,conceptual_tools,Thursday,14:00,15:30
30,2,Ash,problems_2a,Friday,14:00,15:30
33,2,Nicollo,data_science,Friday,15:30,17:00
